# 🧬 molGPT: Conditional Molecular Generation with Transformers

**molGPT** is an end-to-end pipeline for generating novel drug-like molecules using a transformer-based language model (GPT-style). This project focuses on **conditional SMILES generation**, where molecular properties such as LogP, QED, TPSA, and scaffold are used to guide the generation process. It's an exciting intersection of **natural language processing** and **computational drug discovery**.

---

## 🚀 Highlights

- ✅ Trains a decoder-only transformer (GPT) to generate valid SMILES strings
- 🎯 Conditioned on molecular properties like:
  - **LogP** (lipophilicity)
  - **QED** (quantitative estimate of drug-likeness)
  - **TPSA** (topological polar surface area)
  - **Scaffold** (molecular backbone)
- 📊 Includes evaluation metrics:
  - SMILES validity
  - Molecular uniqueness
  - Structural novelty (Tanimoto similarity)
  - Property alignment

---

## 🧪 Dataset

Uses the [MOSES](https://github.com/molecularsets/moses) dataset — a curated collection of drug-like molecules, suitable for generative modeling.

---




In [1]:
# Dependencies

!pip install pandas rdkit transformers[torch] accelerate>=0.26.0
!pip install scikit-learn matplotlib tqdm pathos
!pip install torch --index-url https://download.pytorch.org/whl/cu118
# !pip install git+https://github.com/molecularsets/moses.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.
Looking in indexes: https://download.pytorch.org/whl/cu118


## 1. Import packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm
from pathos.multiprocessing import ProcessingPool as Pool
from functools import partial

from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors, QED, rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    GPT2Config,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)

2025-04-23 10:51:53.734325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745405513.959311      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745405514.024374      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 2. Data Loading and Filtering

 We'll use the MOSES dataset, which is a curated set of drug-like molecules specifically
 designed for machine learning applications. It's much smaller than ChEMBL
 (https://www.ebi.ac.uk/chembl/, database: https://chembl.gitbook.io/chembl-interface-documentation/)
 but still contains high-quality, drug-like compounds.

In [3]:
def load_moses_data():
    filtered_path = Path('/kaggle/input/dataset-v1-filtered/dataset_v1_filtered.csv')
    if filtered_path.exists():
      print(f"Loading pre-filtered dataset from {filtered_path}")
      df = pd.read_csv(filtered_path)
      print(f"Loaded {len(df)} pre-filtered drug-like molecules")
      return df
    train_path = Path('/kaggle/input/dataset-v1/dataset_v1.csv')
    print(f"Reading the original dataset from {train_path}")
    df = pd.read_csv(train_path)
    print(df.info)
    print("\nFirst few rows:")
    print(df.head())

    if 'smiles' in df.columns:
      df = df.rename(columns={'smiles': 'SMILES'})
    elif 'SMILES' not in df.columns:
      print("Available columns:", df.columns.tolist())
      raise ValueError("No 'smiles' or 'SMILES' column found in the dataset.")
    smiles_list = df['SMILES'].values
    print(f"\nFound {len(smiles_list)} SMILES strings in the dataset.")

    # Here we begin with filtering
    valid_mols = []
    for smi in tqdm(smiles_list, desc="Validating and filtering SMILES"):
      # Filter 1 ensures that all molecules are chemically and structurally valid
      mol = Chem.MolFromSmiles(smi)
      if mol is not None:
        # Filter 2 makes sure molecules with physicochemical properties out of a desirable range are removed from the list
        mw = Descriptors.ExactMolWt(mol) # Molecular weight (in Da units)
        logp = Descriptors.MolLogP(mol) # LogP(measured lipophilicity, i.e., how much a molecule likes to be solved in fat versus water)
        hbd = rdMolDescriptors.CalcNumHBD(mol) # Number of a molecule's hydrogen-bond donor heavy atoms
        hba = rdMolDescriptors.CalcNumHBA(mol) # Number of a molecule's hydrogen-bond acceptor heavy atoms

        if mw <= 500 and logp <=5 and hbd <= 5 and hba <= 10:
          # Filter 3 screens for problematic chemical groups shown to be associated with toxicity, carcinogenicity, etc.
          has_bad_groups = False
          patt_list = [
              '[N+]([O-])=O',  # Nitro groups: Highly reactive, can cause DNA damage and carcinogenicity
              '[S](=[O])(=[O])',  # Sulfonyl groups: Can be chemically reactive and cause skin/eye irritation
              '[P](=[O])',  # Phosphoryl groups: Potential toxicity and instability in biological systems
              '[As]'  # Arsenic: Highly toxic heavy metal with severe health risks and carcinogenic properties
          ]
          for patt in patt_list:
              if mol.HasSubstructMatch(Chem.MolFromSmarts(patt)):
                has_bad_groups = True
                break
          if not has_bad_groups:
            valid_mols.append(smi)

    # Create the filtered dataframe
    filtered_df = df[df['SMILES'].isin(valid_mols)]
    print(f"\nAfter filtering, {len(filtered_df)} molecules remain")
    return filtered_df

# Load and display filtered data
filtered_df = load_moses_data()
print("\nFirst few rows of filtered dataset:")
print(filtered_df.head())



Loading pre-filtered dataset from /kaggle/input/dataset-v1-filtered/dataset_v1_filtered.csv
Loaded 1735494 pre-filtered drug-like molecules

First few rows of filtered dataset:
                                   SMILES  SPLIT
0  CCCS(=O)c1ccc2[nH]c(=NC(=O)OC)[nH]c2c1  train
1    CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1  train
2  CC1C2CCC(C2)C1CN(CCO)C(=O)c1ccc(Cl)cc1   test
3     Cc1c(Cl)cccc1Nc1ncccc1C(=O)OCC(O)CO  train
4        Cn1cnc2c1c(=O)n(CC(O)CO)c(=O)n2C  train


## 3. Descriptor Calculation (Scaffolds, logP, QED, TPSA)
 We compute additional descriptors needed:
 - Murcko Scaffolds: Core molecular framework obtained by removing side chains and keeping only ring systems and linkers between rings
 - QED
 - TPSA
 - LogP

We'll store these in the DataFrame alongside the SMILES.

In [4]:
def calculate_descriptors(smiles: str):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None, None, None, None

    try:
        scaffold = GetScaffoldForMol(mol)
        scaffold_smiles = Chem.MolToSmiles(scaffold)

        qed_val = QED.qed(mol)

        tpsa_val = rdMolDescriptors.CalcTPSA(mol)

        logp_val = Descriptors.MolLogP(mol)

        return scaffold_smiles, logp_val, qed_val, tpsa_val
    except:
        return None, None, None, None

def process_batch(smiles_batch):
    results = []
    for smi in smiles_batch:
        results.append(calculate_descriptors(smi))
    return results

def calculate_descriptors_parallel(df, parallel=True, batch_size=100):
    print("Calculating molecular descriptors...")

    if parallel:
        n_cores = Pool().ncpus
        print(f"Detected {n_cores} CPU cores")
        print(f"Running descriptor calculations in parallel across {n_cores} cores")

        smiles_list = df['SMILES'].tolist()
        n_batches = (len(smiles_list) + batch_size - 1) // batch_size
        batches = [smiles_list[i*batch_size:(i+1)*batch_size]
                  for i in range(n_batches)]

        print(f"Processing {len(smiles_list)} SMILES strings in {n_batches} batches")

        with Pool() as pool:
            results = list(tqdm(
                pool.imap(process_batch, batches),
                total=n_batches,
                desc="Processing batches"
            ))

        all_results = [item for batch in results for item in batch]

    else:
        print("Running descriptor calculations sequentially")
        all_results = []
        for smi in tqdm(df['SMILES'], desc="Calculating descriptors"):
            all_results.append(calculate_descriptors(smi))

    scaffolds, logps, qeds, tpsas = zip(*all_results)

    df['Scaffold'] = scaffolds
    df['LogP'] = logps
    df['QED'] = qeds
    df['TPSA'] = tpsas

    df = df.dropna(subset=['Scaffold', 'LogP', 'QED', 'TPSA'])
    print(f"Final dataset size after descriptor calculation: {len(df)}")

    print("\nDescriptor Statistics:")
    print(f"LogP range: {df['LogP'].min():.2f} to {df['LogP'].max():.2f}")
    print(f"QED range: {df['QED'].min():.2f} to {df['QED'].max():.2f}")
    print(f"TPSA range: {df['TPSA'].min():.2f} to {df['TPSA'].max():.2f}")

    return df

filtered_df = calculate_descriptors_parallel(filtered_df, parallel=True)
filtered_df.head()

Calculating molecular descriptors...
Detected 4 CPU cores
Running descriptor calculations in parallel across 4 cores
Processing 1735494 SMILES strings in 17355 batches


Processing batches:   0%|          | 0/17355 [00:00<?, ?it/s]

Final dataset size after descriptor calculation: 1735494

Descriptor Statistics:
LogP range: -4.16 to 5.00
QED range: 0.21 to 0.95
TPSA range: 0.00 to 206.50


,SMILES,SPLIT,Scaffold,LogP,QED,TPSA
0,CCCS(=O)c1ccc2[nH]c(=NC(=O)OC)[nH]c2c1,train,N=c1[nH]c2ccccc2[nH]1,1.68070,0.896898,87.31
1,CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1,train,c1ccc(OCn2ccnc2)cc1,3.72930,0.862259,44.12
2,CC1C2CCC(C2)C1CN(CCO)C(=O)c1ccc(Cl)cc1,test,O=C(NCC1CC2CCC1C2)c1ccccc1,3.45670,0.901948,40.54
3,Cc1c(Cl)cccc1Nc1ncccc1C(=O)OCC(O)CO,train,c1ccc(Nc2ccccn2)cc1,2.29702,0.701022,91.68
4,Cn1cnc2c1c(=O)n(CC(O)CO)c(=O)n2C,train,O=c1[nH]c(=O)c2[nH]cnc2[nH]1,-2.21310,0.646083,102.28


## 4. Preparing the Data for Conditional Generation

We will train a GPT model to generate the full SMILES given:
1. The scaffold SMILES
2. The desired LogP
3. The desired QED
4. The desired TPSA

One straightforward approach is to serialize these conditions into a single text string.
For example:

    "SCAFFOLD: Cc1ccccn1 | LOGP: 2.3 | QED: 0.72 | TPSA: 32.4 => FULL_SMILES"

We then train the model in a language modeling fashion to predict FULL_SMILES from
these inputs. Alternatively, we could build a more sophisticated approach that
incorporates the numeric data differently, but for demonstration, we'll do it text-based

In [5]:
def create_condition_text(row):
  scaffold_str = row['Scaffold']
  logp_str = f"{row['LogP']:.2f}"
  qed_str = f"{row['QED']:.2f}"
  tpsa_str = f"{row['TPSA']:.1f}"
  full_smiles = row['SMILES']

  input_text = f"SCAFFOLD: {scaffold_str} | LOGP: {logp_str} | QED: {qed_str} | TPSA: {tpsa_str} => f{full_smiles}"
  return input_text

filtered_df['conditional_text'] = filtered_df.apply(create_condition_text, axis=1)
filtered_df.head()

,SMILES,SPLIT,Scaffold,LogP,QED,TPSA,conditional_text
0,CCCS(=O)c1ccc2[nH]c(=NC(=O)OC)[nH]c2c1,train,N=c1[nH]c2ccccc2[nH]1,1.68070,0.896898,87.31,SCAFFOLD: N=c1[nH]c2ccccc2[nH]1 | LOGP: 1.68 |...
1,CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1,train,c1ccc(OCn2ccnc2)cc1,3.72930,0.862259,44.12,SCAFFOLD: c1ccc(OCn2ccnc2)cc1 | LOGP: 3.73 | Q...
2,CC1C2CCC(C2)C1CN(CCO)C(=O)c1ccc(Cl)cc1,test,O=C(NCC1CC2CCC1C2)c1ccccc1,3.45670,0.901948,40.54,SCAFFOLD: O=C(NCC1CC2CCC1C2)c1ccccc1 | LOGP: 3...
3,Cc1c(Cl)cccc1Nc1ncccc1C(=O)OCC(O)CO,train,c1ccc(Nc2ccccn2)cc1,2.29702,0.701022,91.68,SCAFFOLD: c1ccc(Nc2ccccn2)cc1 | LOGP: 2.30 | Q...
4,Cn1cnc2c1c(=O)n(CC(O)CO)c(=O)n2C,train,O=c1[nH]c(=O)c2[nH]cnc2[nH]1,-2.21310,0.646083,102.28,SCAFFOLD: O=c1[nH]c(=O)c2[nH]cnc2[nH]1 | LOGP:...


## 5. Train/Validation/Test Split

In [6]:
from sklearn.model_selection import train_test_split

train_df, valtest_df = train_test_split(filtered_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(valtest_df, test_size=0.5, random_state=42)

print(f"Train size: {len(train_df)}")
print(f"Val size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 1388395
Val size: 173549
Test size: 173550


## 6. Defining the GPT Model and Tokenizer
We'll create a small GPT2 model from scratch (or you could fine-tune a pretrained GPT2).
For demonstration, we'll use a smaller config for faster training.

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=256,
    n_ctx=256,
    n_embd=128,
    n_layer=4,
    n_head=4
)


model = GPT2LMHeadModel(config)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

## 7. Create a Custom Dataset for Language Modeling

In [8]:
class SmilesConditionalDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=256):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        txt = self.data.iloc[idx]['conditional_text']

        return self.tokenizer(
            txt,
            max_length=self.max_length,
            truncation=True,
            return_special_tokens_mask=True
        )


train_dataset = SmilesConditionalDataset(train_df, tokenizer)
val_dataset   = SmilesConditionalDataset(val_df, tokenizer)
test_dataset  = SmilesConditionalDataset(test_df, tokenizer)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

## 8. Training the model

In [9]:
# training_args = TrainingArguments(
#     output_dir="./conditional_gpt_smiles",
#     overwrite_output_dir=True,
#     num_train_epochs=1,
#     per_device_train_batch_size=2,
#     per_device_eval_batch_size=2,
#     eval_strategy="steps",
#     eval_steps=50,
#     save_steps=50,
#     logging_steps=50,
#     save_total_limit=1,
#     learning_rate=1e-4,
#     warmup_steps=100,
#     weight_decay=0.01,
#     run_name="gpt-smiles-cond-epoch1",                # <--- 避免 W&B 的 warning
#     report_to="none",               # <--- 启用 wandb 集成（自动记录 metrics）
 
# )

training_args = TrainingArguments(
    output_dir="./conditional_gpt_smiles",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,        # Increased
    per_device_eval_batch_size=16,         # Increased
    gradient_accumulation_steps=8,        # Added
    fp16=True,                            # Added for mixed precision
    eval_strategy="steps",
    eval_steps=1500,                       # Reduced frequency
    save_steps=1500,                       # Reduced frequency
    logging_steps=1200,                    # Reduced frequency
    save_total_limit=1,
    learning_rate=1e-3,                   # Adjusted for larger batch
    warmup_steps=100,
    weight_decay=0.01,
    dataloader_num_workers=4,             # Added
    run_name="gpt-smiles-cond-epoch1",               
    report_to="none",
)

# training_args = TrainingArguments(
#     output_dir="./conditional_gpt_smiles",
#     overwrite_output_dir=True,
#     num_train_epochs=1,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     gradient_accumulation_steps=8,
#     fp16=True,
#     fp16_opt_level="O2",           # More aggressive mixed precision
#     eval_strategy="steps",
#     eval_steps=500,
#     save_steps=500,
#     logging_steps=200,
#     save_total_limit=1,
#     learning_rate=1e-3,            # Adjusted for larger effective batch
#     warmup_steps=100,
#     weight_decay=0.01,
#     dataloader_num_workers=8,
#     dataloader_pin_memory=True,    # Faster data transfer
#     gradient_checkpointing=True,   # Memory efficient training
#     run_name="gpt-smiles-cond-epoch1",
#     report_to="none",
# )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
print("Starting training...")
trainer.train()

Starting training...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
1500,1.292600,0.555617
3000,0.584600,0.493483
4500,0.533900,0.464609
6000,0.490300,0.447951
7500,0.477500,0.436301
9000,0.467700,0.425407
10500,0.458900,0.419265
12000,0.447200,0.413480
13500,0.442000,0.407734
15000,0.437800,0.403773


TrainOutput(global_step=32538, training_loss=0.47483191506879785, metrics={'train_runtime': 11973.771, 'train_samples_per_second': 347.859, 'train_steps_per_second': 2.717, 'total_flos': 1905046386204672.0, 'train_loss': 0.47483191506879785, 'epoch': 2.999919331604725})

## 9. Inference (Generating SMILES from Conditions)

During inference, we'll provide the scaffold and desired property values. For example:

"Scaffold: <scaffold> | LogP: <val> | QED: <val> | TPSA: <val> =>"

The model should complete the sequence by generating a SMILES string.

We'll generate multiple samples to test uniqueness and validity.

In [11]:

# def generate_smiles_from_conditions(model, tokenizer, scaffold, logp, qed, tpsa,
#                                     max_length=256, num_return_sequences=1):
#     prompt = f"Scaffold: {scaffold} | LogP: {logp:.2f} | QED: {qed:.2f} | TPSA: {tpsa:.2f} =>"

#     input_ids = tokenizer.encode(prompt, return_tensors='pt')
#     input_ids = input_ids.to(model.device)

#     with torch.no_grad():
#         outputs = model.generate(
#             input_ids=input_ids,
#             max_length=max_length,
#             num_return_sequences=num_return_sequences,
#             do_sample=True,           # Use sampling
#             top_k=50,                 # Adjust as desired
#             top_p=0.95,               # Adjust as desired
#             temperature=0.7,          # Adjust as desired
#             pad_token_id=tokenizer.eos_token_id
#         )

#     generated_texts = []
#     for output in outputs:
#         text = tokenizer.decode(output, skip_special_tokens=True)
#         # We want only the SMILES part after the "=>"
#         if "=>" in text:
#             smiles_part = text.split("=>")[-1].strip()
#             generated_texts.append(smiles_part)
#         else:
#             generated_texts.append(text)

#     return generated_texts

# trainer.model.eval()
# row = test_df.iloc[0]
# generated_smiles = generate_smiles_from_conditions(
#     model=trainer.model,
#     tokenizer=tokenizer,
#     scaffold=row['Scaffold'],
#     logp=row['LogP'],
#     qed=row['QED'],
#     tpsa=row['TPSA'],
#     num_return_sequences=5
# )
# # print(generated_smiles)


def generate_smiles_from_conditions(model, tokenizer, scaffold, logp, qed, tpsa,
                                    max_length=256, num_return_sequences=1):
    prompt = f"SCAFFOLD: {scaffold} | LOGP: {logp:.2f} | QED: {qed:.2f} | TPSA: {tpsa:.2f} =>"

    # 显式创建注意力掩码
    encoded_input = tokenizer(prompt, return_tensors='pt', padding=True)
    input_ids = encoded_input['input_ids'].to(model.device)
    attention_mask = encoded_input['attention_mask'].to(model.device)
    
    print(f"生成条件: {prompt}")
    print(f"输入序列长度: {input_ids.shape}")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=True,           # Use sampling
            top_k=100,                # Increased from 50 to 100
            top_p=0.9,                # Lowered from 0.95 to 0.9
            temperature=0.5,          # Lowered from 0.7 to 0.5
            pad_token_id=tokenizer.eos_token_id
        )
        #     print("模型生成完成")
        # except Exception as e:
        #     print(f"生成过程中发生错误: {e}")
        #     return []

    generated_texts = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        print(f"原始生成文本: {text}")
        # 我们只需要 "=>" 后面的SMILES部分
        if "=>" in text:
            smiles_part = text.split("=>")[-1].strip()
            generated_texts.append(smiles_part)
        else:
            generated_texts.append(text)

    return generated_texts

# 添加调试信息
print("准备评估模型...")
trainer.model.eval()
print("模型已设置为评估模式")

# try:
#     row = test_df.iloc[0]
#     print(f"测试样本: Scaffold={row['Scaffold']}, LogP={row['LogP']:.2f}, QED={row['QED']:.2f}, TPSA={row['TPSA']:.2f}")
    
#     generated_smiles = generate_smiles_from_conditions(
#         model=trainer.model,
#         tokenizer=tokenizer,
#         scaffold=row['Scaffold'],
#         logp=row['LogP'],
#         qed=row['QED'],
#         tpsa=row['TPSA'],
#         num_return_sequences=1  # 先尝试只生成一个样本
#     )
    
#     print("生成结果:")
#     for i, smi in enumerate(generated_smiles):
#         print(f"生成的SMILES {i+1}: {smi}")
#         mol = Chem.MolFromSmiles(smi)
#         if mol:
#             print(f"有效的SMILES: 是")
#         else:
#             print(f"有效的SMILES: 否")
# except Exception as e:
#     print(f"执行过程中发生错误: {str(e)}")



准备评估模型...
模型已设置为评估模式


## 10. Evaluation
We will:
1. Generate 1000 molecules with random conditions from the test set.
2. Check:
   - Valid SMILES: Can RDKit parse them?
   - Unique SMILES: How many are duplicates?
   - Tanimoto similarity to training set.
   - Distribution of predicted properties.

In [12]:
def is_valid_smiles(smi):
    mol = Chem.MolFromSmiles(smi)
    return mol is not None

def compute_tanimoto_similarity(smi1, smi2, radius=2, nBits=2048):
    mol1 = Chem.MolFromSmiles(smi1)
    mol2 = Chem.MolFromSmiles(smi2)
    if mol1 is None or mol2 is None:
        return None
    fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, radius, nBits=nBits)
    fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, radius, nBits=nBits)
    return rdMolDescriptors.TanimotoSimilarity(fp1, fp2)

# Let's define a quick function to evaluate.
# In practice, you might do more robust analysis.
def evaluate_model(
    model,
    tokenizer,
    reference_df,
    n_samples=1000
):
    model.eval()

    valid_count = 0
    unique_smiles = set()
    similarities = []

    # We'll store the property differences if we want to check property distribution.
    requested_logps, generated_logps = [], []
    requested_qeds, generated_qeds = [], []
    requested_tpsas, generated_tpsas = [], []

    # Convert train_df SMILES to a list for similarity reference
    train_smiles_list = train_df['SMILES'].tolist()

    for i in range(n_samples):
        # Randomly select a row from the reference set or sample property values
        row = reference_df.sample(n=1).iloc[0]
        scaffold = row['Scaffold']
        logp_req = row['LogP']
        qed_req = row['QED']
        tpsa_req = row['TPSA']

        gen_smiles_list = generate_smiles_from_conditions(
            model, tokenizer, scaffold, logp_req, qed_req, tpsa_req,
            num_return_sequences=1
        )

        gen_smi = gen_smiles_list[0]

        if is_valid_smiles(gen_smi):
            valid_count += 1
            unique_smiles.add(Chem.MolToSmiles(Chem.MolFromSmiles(gen_smi)))  # canonical

            # Tanimoto similarity (just to the original training data)
            # We'll compute the max similarity to any molecule in the training set
            # as a measure of novelty.
            best_sim = 0
            for train_smi in train_smiles_list[:1000]:  # limit to 1000 for speed
                sim = compute_tanimoto_similarity(gen_smi, train_smi)
                if sim is not None and sim > best_sim:
                    best_sim = sim
            similarities.append(best_sim)

            # Check the property distribution if desired
            gen_mol = Chem.MolFromSmiles(gen_smi)
            if gen_mol:
                gen_logp = Descriptors.MolLogP(gen_mol)
                gen_qed = QED.qed(gen_mol)
                gen_tpsa = Descriptors.TPSA(gen_mol)

                requested_logps.append(logp_req)
                generated_logps.append(gen_logp)
                requested_qeds.append(qed_req)
                generated_qeds.append(gen_qed)
                requested_tpsas.append(tpsa_req)
                generated_tpsas.append(gen_tpsa)

    validity_ratio = valid_count / n_samples
    uniqueness_ratio = len(unique_smiles) / n_samples
    avg_similarity = np.mean(similarities) if similarities else 0

    print(f"Validity: {validity_ratio:.2f}")
    print(f"Uniqueness: {uniqueness_ratio:.2f}")
    print(f"Average Tanimoto similarity to training set: {avg_similarity:.2f}")

    # Plot property distribution comparisons
    # For example, requested vs generated LogP
    if len(requested_logps) > 0:
        plt.figure()
        plt.scatter(requested_logps, generated_logps, alpha=0.5)
        plt.xlabel("Requested LogP")
        plt.ylabel("Generated LogP")
        plt.title("Requested vs. Generated LogP")
        plt.show()

        plt.figure()
        plt.scatter(requested_qeds, generated_qeds, alpha=0.5)
        plt.xlabel("Requested QED")
        plt.ylabel("Generated QED")
        plt.title("Requested vs. Generated QED")
        plt.show()

        plt.figure()
        plt.scatter(requested_tpsas, generated_tpsas, alpha=0.5)
        plt.xlabel("Requested TPSA")
        plt.ylabel("Generated TPSA")
        plt.title("Requested vs. Generated TPSA")
        plt.show()

In [13]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, QED, rdMolDescriptors
import torch

# Assuming you have already loaded or have access to:
# 1. The trained model (model)
# 2. The tokenizer (tokenizer)
# 3. The test dataset (test_df)
# 4. The training dataset (train_df)

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Run the evaluation with 1000 samples
print("Starting model evaluation with 1000 samples...")
evaluate_model(
    model=model,
    tokenizer=tokenizer,
    reference_df=test_df,
    n_samples=10
)

# If you want to run with fewer samples for a quicker test
print("\nQuick evaluation with 100 samples...")
evaluate_model(
    model=model,
    tokenizer=tokenizer,
    reference_df=test_df,
    n_samples=10
)

# Optional: Save some example molecules as images
def visualize_examples(model, tokenizer, test_df, num_examples=5):
    print(f"\nGenerating {num_examples} example molecules...")
    example_rows = test_df.sample(n=num_examples)
    
    for i, row in enumerate(example_rows.itertuples()):
        scaffold = row.Scaffold
        logp_req = row.LogP
        qed_req = row.QED
        tpsa_req = row.TPSA
        
        print(f"\nExample {i+1}:")
        print(f"Scaffold: {scaffold}")
        print(f"Requested LogP: {logp_req:.2f}")
        print(f"Requested QED: {qed_req:.2f}")
        print(f"Requested TPSA: {tpsa_req:.2f}")
        
        gen_smiles_list = generate_smiles_from_conditions(
            model, tokenizer, scaffold, logp_req, qed_req, tpsa_req,
            num_return_sequences=3
        )
        
        for j, smi in enumerate(gen_smiles_list):
            if is_valid_smiles(smi):
                mol = Chem.MolFromSmiles(smi)
                print(f"  Generated SMILES {j+1}: {smi}")
                
                # Calculate actual properties
                gen_logp = Descriptors.MolLogP(mol)
                gen_qed = QED.qed(mol)
                gen_tpsa = Descriptors.TPSA(mol)
                
                print(f"  Actual LogP: {gen_logp:.2f} (diff: {gen_logp - logp_req:.2f})")
                print(f"  Actual QED: {gen_qed:.2f} (diff: {gen_qed - qed_req:.2f})")
                print(f"  Actual TPSA: {gen_tpsa:.2f} (diff: {gen_tpsa - tpsa_req:.2f})")
            else:
                print(f"  Generated SMILES {j+1}: {smi} (INVALID)")

# Run the visualization function
visualize_examples(model, tokenizer, test_df, num_examples=5)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Starting model evaluation with 1000 samples...
生成条件: SCAFFOLD: c1ccccc1 | LOGP: 2.35 | QED: 0.81 | TPSA: 38.77 =>
输入序列长度: torch.Size([1, 34])
原始生成文本: SCAFFOLD: c1ccccc1 | LOGP: 2.35 | QED: 0.81 | TPSA: 38.77 => fCOC(=O)C(C)Nc1cccc(C(F)(F)F)c1F)C(C)C11C=O | TPSA: 81.2)ccc1F)O)N1CCO)C1)C111C1 | LOGP: TPSA:1)cccccc1)F)N)O)C#N)C1C1F)N)C1F)C1C#N1C#N1F)C#N1F)C#N1F)C#N1C#N1C#N1C#N1)C#N1F)C#N1F)C#N1F)C#N1C#N1F)C#N1F)C#N1F)C#N1F)C#N1F)C#N1F)C#N1F)C#N1FF#N1F)
生成条件: SCAFFOLD: O=C(NC1CCCCC1)c1cccnn1 | LOGP: 2.10 | QED: 0.91 | TPSA: 72.11 =>
输入序列长度: torch.Size([1, 44])


[14:54:01] SMILES Parse Error: syntax error while parsing: fCOC(=O)C(C)Nc1cccc(C(F)(F)F)c1F)C(C)C11C=O
[14:54:01] SMILES Parse Error: check for mistakes around position 1:
[14:54:01] fCOC(=O)C(C)Nc1cccc(C(F)(F)F)c1F)C(C)C11C
[14:54:01] ^
[14:54:01] SMILES Parse Error: Failed parsing SMILES 'fCOC(=O)C(C)Nc1cccc(C(F)(F)F)c1F)C(C)C11C=O' for input: 'fCOC(=O)C(C)Nc1cccc(C(F)(F)F)c1F)C(C)C11C=O'


原始生成文本: SCAFFOLD: O=C(NC1CCCCC1)c1cccnn1 | LOGP: 2.10 | QED: 0.91 | TPSA: 72.11 => fCC(=O)Nc1ccc(C(=O)NC2CCCCC2)nn1C(C)C)C1CC1F1 | TPSA:2)F11111111111111111111111111 | LOGP: 0.4 | T1)1)nn1)C11)C11)C11114444444444444421 | TPS3142 | TPS3142 | TPS42 | T42 | T4CO4CO4CO42CO42C3CO4CO4CO4CO4C2CO4CO4CO4CO4CO4CO4CO4CO4C2C2C2CO2C2CO4C1C2C2C2CO2CO1CO1CO4C2C2CO2C
生成条件: SCAFFOLD: O=C(Cc1cnoc1)N1CCC(Oc2ccccc2)CC1 | LOGP: 3.52 | QED: 0.85 | TPSA: 55.57 =>
输入序列长度: torch.Size([1, 53])


[14:54:02] SMILES Parse Error: syntax error while parsing: fCC(=O)Nc1ccc(C(=O)NC2CCCCC2)nn1C(C)C)C1CC1F1
[14:54:02] SMILES Parse Error: check for mistakes around position 1:
[14:54:02] fCC(=O)Nc1ccc(C(=O)NC2CCCCC2)nn1C(C)C)C1C
[14:54:02] ^
[14:54:02] SMILES Parse Error: Failed parsing SMILES 'fCC(=O)Nc1ccc(C(=O)NC2CCCCC2)nn1C(C)C)C1CC1F1' for input: 'fCC(=O)Nc1ccc(C(=O)NC2CCCCC2)nn1C(C)C)C1CC1F1'


...

[14:54:26] SMILES Parse Error: syntax error while parsing: fCC(C)(C)OC(=O)Nc1nc(-c2ccccc2)cs1
[14:54:26] SMILES Parse Error: check for mistakes around position 1:
[14:54:26] fCC(C)(C)OC(=O)Nc1nc(-c2ccccc2)cs1
[14:54:26] ^
[14:54:26] SMILES Parse Error: Failed parsing SMILES 'fCC(C)(C)OC(=O)Nc1nc(-c2ccccc2)cs1' for input: 'fCC(C)(C)OC(=O)Nc1nc(-c2ccccc2)cs1'
[14:54:26] SMILES Parse Error: syntax error while parsing: fCC(C)C(=O)Nc1nc(-c2ccc(Cl)cc2)cs1
[14:54:26] SMILES Parse Error: check for mistakes around position 1:
[14:54:26] fCC(C)C(=O)Nc1nc(-c2ccc(Cl)cc2)cs1
[14:54:26] ^
[14:54:26] SMILES Parse Error: Failed parsing SMILES 'fCC(C)C(=O)Nc1nc(-c2ccc(Cl)cc2)cs1' for input: 'fCC(C)C(=O)Nc1nc(-c2ccc(Cl)cc2)cs1'
[14:54:26] SMILES Parse Error: syntax error while parsing: fCC(=O)Nc1nc(-c2ccc(NC(=O)CC(C)C)cc2)cs1
[14:54:26] SMILES Parse Error: check for mistakes around position 1:
[14:54:26] fCC(=O)Nc1nc(-c2ccc(NC(=O)CC(C)C)cc2)cs1
[14:54:26] ^
[14:54:26] SMILES Parse Error: Failed parsing 

原始生成文本: SCAFFOLD: O=C1CCN(CCC(=O)c2ccccc2)CC1 | LOGP: 2.55 | QED: 0.78 | TPSA: 46.61 => fO=C1CCN(CCC(=O)c2ccc(Cl)cc2)CC1=O | TPSA: 75.4)CC1C1C1C1C1C1C1C1C1C1C1C1 | LOGP:C1C1 | TPSA: 2.6 | TPSA:2 | TPSA:2=O4)C1C1C1)C1)C1C1)C1C1C1C1C1)C1C1C1C1C1C1)C1C1)C1C1C1)C1(N)=O)=O4C1C1(C#N)=O4C#N)=O)=O4N)=O4C#N)=O4C#N)=O4O4O4O4CO4O1O4O4O4O4O1
原始生成文本: SCAFFOLD: O=C1CCN(CCC(=O)c2ccccc2)CC1 | LOGP: 2.55 | QED: 0.78 | TPSA: 46.61 => fC=CCOc1cccc(C(=O)CCN2CCC(=O)CC2)c1F)C1CC1C)C1C)C1C1C1 | TPSA: | TPSA:2)C1C1)C1C1C1C1C1C1)C1C1)C1C1)C1C1C1C1C1C1C1)C1C1C1C1C1)C1)C1C1C1C1C1C11C1)C11C1C1C1)C1C11C1C1C1)C1 | LOGP: | LOGP: | LOGP::::1C1C1C1C1 | LOGP::1C1C1C1C1C1C1C1C1)C1)C1
原始生成文本: SCAFFOLD: O=C1CCN(CCC(=O)c2ccccc2)CC1 | LOGP: 2.55 | QED: 0.78 | TPSA: 46.61 => fC=CCOc1ccc(C(=O)CCN2CCC(=O)CC2)cc1F)cc1F | TPSA:FFOLD:FFOLD:)C1CC1 | TPSA:)C1CC1 | T1 | T1)cc1)F)nn1C1C1)N)C1C1C1)C1CC1)C1C1)C1C1C1C1C1C1C1C1)N1C1C1C1C1)C1C1C1C1 | TPSA:1C1C1O1C1C1C1C1C1C1C1C1C1C1C1)C1 | LOGP:1 | LOGP:1C1 | LOGP:1C1C1C1C1O1 | LOGP:1C1C1

[14:54:28] SMILES Parse Error: syntax error while parsing: fO=C1CCN(CCC(=O)c2ccc(Cl)cc2)CC1=O
[14:54:28] SMILES Parse Error: check for mistakes around position 1:
[14:54:28] fO=C1CCN(CCC(=O)c2ccc(Cl)cc2)CC1=O
[14:54:28] ^
[14:54:28] SMILES Parse Error: Failed parsing SMILES 'fO=C1CCN(CCC(=O)c2ccc(Cl)cc2)CC1=O' for input: 'fO=C1CCN(CCC(=O)c2ccc(Cl)cc2)CC1=O'
[14:54:28] SMILES Parse Error: syntax error while parsing: fC=CCOc1cccc(C(=O)CCN2CCC(=O)CC2)c1F)C1CC1C)C1C)C1C1C1
[14:54:28] SMILES Parse Error: check for mistakes around position 1:
[14:54:28] fC=CCOc1cccc(C(=O)CCN2CCC(=O)CC2)c1F)C1CC
[14:54:28] ^
[14:54:28] SMILES Parse Error: Failed parsing SMILES 'fC=CCOc1cccc(C(=O)CCN2CCC(=O)CC2)c1F)C1CC1C)C1C)C1C1C1' for input: 'fC=CCOc1cccc(C(=O)CCN2CCC(=O)CC2)c1F)C1CC1C)C1C)C1C1C1'
[14:54:28] SMILES Parse Error: syntax error while parsing: fC=CCOc1ccc(C(=O)CCN2CCC(=O)CC2)cc1F)cc1F
[14:54:28] SMILES Parse Error: check for mistakes around position 1:
[14:54:28] fC=CCOc1ccc(C(=O)CCN2CCC(=O)CC2)

原始生成文本: SCAFFOLD: O=C(Oc1ccccc1)C1CC(=O)N(Cc2ccco2)C1 | LOGP: 2.80 | QED: 0.63 | TPSA: 59.75 => fCc1ccc(C)c(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c1C(=O)N(C)C)C1C2C1C1C1C1C1C1C111C11C1C1C11C11C1C11C1C1C1C1C11C1C1C1C1C1C1111C1C11C11111C111C1111 | LOGP: 0. | LOGP: 0.96 | TPSA: 0.76 | TPSA: 0. | TPSA: 0.62 | TPSA:: 3. | TPSA:2 | TPSA:1C1C11111C1
原始生成文本: SCAFFOLD: O=C(Oc1ccccc1)C1CC(=O)N(Cc2ccco2)C1 | LOGP: 2.80 | QED: 0.63 | TPSA: 59.75 => fCc1ccc(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c(C)c1C)C1C1CC1 | TPSA:)O1)O1)C1)C11)C11)C1111F)C1C1C1F)O1C111F)N1F)C1F)C1F)N1F)C1F)C1C1F)C1F)C1F)C1F)C1F1C1C1C1F)C1FF1C1FF4O4)C1C1C1C1CC1F)C1C1F1C1C1F4O4O4O4O1F)N1C1C1C1F)
原始生成文本: SCAFFOLD: O=C(Oc1ccccc1)C1CC(=O)N(Cc2ccco2)C1 | LOGP: 2.80 | QED: 0.63 | TPSA: 59.75 => fO=C(Oc1cccc(Cl)c1)C1CC(=O)N(Cc2ccco2)C1C1CC1 | TPSA: TPSA: 75.66111 | TPSA: 75.71)C1)C11)OCC1)N1)N11111F)N111F)C111111F)C1F)C1C111111111111F)N1C1C1F)C11C11C11C1C1C1C1CO1C1CO1CO1C1C1C1CO1CO1CO1CO1C1C1CO1CO1CO1CO1CO1CO1CO1CO1CO1CO1C1CO1CO
  Generated SMILES 1: fCc1

[14:54:29] SMILES Parse Error: syntax error while parsing: fCc1ccc(C)c(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c1C(=O)N(C)C)C1C2C1C1C1C1C1C1C111C11C1C1C11C11C1C11C1C1C1C1C11C1C1C1C1C1C1111C1C11C11111C111C1111
[14:54:29] SMILES Parse Error: check for mistakes around position 1:
[14:54:29] fCc1ccc(C)c(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c
[14:54:29] ^
[14:54:29] SMILES Parse Error: Failed parsing SMILES 'fCc1ccc(C)c(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c1C(=O)N(C)C)C1C2C1C1C1C1C1C1C111C11C1C1C11C11C1C11C1C1C1C1C11C1C1C1C1C1C1111C1C11C11111C111C1111' for input: 'fCc1ccc(C)c(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c1C(=O)N(C)C)C1C2C1C1C1C1C1C1C111C11C1C1C11C11C1C11C1C1C1C1C11C1C1C1C1C1C1111C1C11C11111C111C1111'
[14:54:29] SMILES Parse Error: syntax error while parsing: fCc1ccc(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c(C)c1C)C1C1CC1
[14:54:29] SMILES Parse Error: check for mistakes around position 1:
[14:54:29] fCc1ccc(OC(=O)C2CC(=O)N(Cc3ccco3)C2)c(C)c
[14:54:29] ^
[14:54:29] SMILES Parse Error: Failed parsing SMILES 'fCc1ccc(OC(=O)C2CC(=O)N(